## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
%run ./includes/utilities

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, input_file_name, current_timestamp, from_json, expr, explode, split, to_timestamp, regexp_replace
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType, IntegerType
from transformers import pipeline
import mlflow.pyfunc
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import boto3  
from transformers import BertTokenizer  
from delta.tables import DeltaTable
import mlflow
import seaborn as sns


In [0]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Tweet Processing") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()


- Are your shuffle partitions consistent with your cluster and your workload?

    Solution: With a cluster of 4 cores, we need number of partitions equal to multiple of 4. Setting Spark shuffle partitions to 8 allows for double the number of partitions relative to your cores. This can help improve parallelism and better utilize each core, especially during operations that involve data shuffling. 


- Do you have the necessary libraries to perform the required operations in the pipeline/application?

    Solution: I have imported the necessary libraries to perform the required operations in the pipeline.

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# Reading the source file directory listing
source_files_df = get_source_listing_df()
file_count = source_files_df.count()

In [0]:

# Counting the source files 
print(f"Number of source files: {file_count}")

In [0]:
print("Columns available in DataFrame:", source_files_df.columns)


In [0]:
# Step 3: Print the content of one file 
first_file_path = source_files_df.head(1)['File Name'].values[0]
show_s3_file_contents(first_file_path)

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Define the schema for the raw tweet data
tweet_schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True)
])

print("Tweet schema defined.")

In [0]:
# Read stream from CloudFiles, applying the defined schema
raw_tweet_df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .schema(tweet_schema)
    .load(TWEET_SOURCE_PATH)
)

print("Read stream for raw tweet data setup.")

In [0]:
display(raw_tweet_df, truncate = False)

In [0]:
# Add the new columns to the DataFrame
bronze_tweet_df = (
    raw_tweet_df
    .withColumn("source_file", input_file_name())  # Extract file path
    .withColumn("processing_time", current_timestamp())  # Current timestamp
)

print("Transformation to Bronze data completed.")

In [0]:
display(bronze_tweet_df, truncate = False)

In [0]:
# Write the stream to the Bronze Delta table, allowing schema evolution
bronze_stream = (
    bronze_tweet_df
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", BRONZE_CHECKPOINT)
    .option("mergeSchema", "true")  # Allow new schema to be merged
    .trigger(availableNow = True)
    .start(BRONZE_DELTA)
)

print("Write stream to Bronze Delta table started as bronze_stream.")

In [0]:
bronze_stream.awaitTermination()

In [0]:
bronze_stream.status

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
  
  Solution: 200000
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
  
  Solution: There are no columns that contain null values.
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
  
  Solution: 93817 tweets have at least one mention and 106183 have no mentions.
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
spark.readStream.load(BRONZE_DELTA).createOrReplaceTempView('bronze_temp')

In [0]:
%sql
-- # ENTER YOUR CODE HERE
SELECT * FROM bronze_temp

In [0]:
%sql
SELECT COUNT(*) FROM bronze_temp

In [0]:
%sql
SELECT
  COUNT(IF(date IS NULL, 1, NULL)) AS missing_dates, 
  COUNT(IF(user IS NULL, 1, NULL)) AS missing_users, 
  COUNT(IF(text IS NULL, 1, NULL)) AS missing_texts,
  COUNT(IF(source_file IS NULL, 1, NULL)) AS missing_source_file,
  COUNT(IF(sentiment IS NULL, 1, NULL)) AS missing_sentiments
FROM bronze_temp

In [0]:
%sql
SELECT user, COUNT(*) as total_tweets
FROM bronze_temp
GROUP BY user
ORDER BY total_tweets DESC

In [0]:
%sql
SELECT 
  SUM(CASE WHEN text LIKE '%@%' THEN 1 ELSE 0 END) AS tweets_with_mention,
  SUM(CASE WHEN text NOT LIKE '%@%' THEN 1 ELSE 0 END) AS tweets_without_mention
FROM bronze_temp;


In [0]:
%sql
SELECT user, COUNT(*) as total_tweets
FROM bronze_temp
GROUP BY user
ORDER BY total_tweets DESC
LIMIT 20

Databricks visualization. Run in Databricks to view.

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# ENTER YOUR CODE HERE
# Read stream from the Bronze Delta table
bronze_df_stream = (
    spark.readStream
    .format("delta")
    .load(BRONZE_DELTA)
)

print("Read stream from Bronze Delta table setup.")

In [0]:
display(bronze_df_stream)

In [0]:
# UDF to extract mentions starting with '@' from text
extract_mentions = udf(lambda text: [word[1:] for word in text.split() if word.startswith('@') and len(word) > 1], ArrayType(StringType()))


In [0]:
# UDF to extract cleaned text
extract_cleaned_text = udf(lambda text: ' '.join([word for word in text.split() if not word.startswith('@')]))


In [0]:
# Filtering rows with having atleast one mentions
# Adding 'mention' and 'cleaned_text' columns using udf
bronze_to_silver = (bronze_df_stream
    .select("date", "text", "sentiment")
    .filter(col("text").like("%@%"))
    .withColumn("mention_array", extract_mentions(col("text")))
    .withColumn("cleaned_text", extract_cleaned_text(col("text")))
)


In [0]:
# Explode the mention array column to get the row for each mention
bronze_to_silver = (bronze_to_silver
    .select("*", explode("mention_array").alias("mention"))
)



In [0]:
# Adding timestamp column using date column
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
bronze_to_silver = bronze_to_silver.withColumn("timestamp", to_timestamp(col("date"), "EEE MMM dd HH:mm:ss z yyyy"))


In [0]:
# Dropping the redundant columns
bronze_to_silver = bronze_to_silver.drop("date", "text", "mention_array")

In [0]:
# Write the transformed data to the Silver Delta table
silver_stream = (
    bronze_to_silver
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("mergeSchema", "true")  # Allow new schema to be merged
    .option("checkpointLocation", SILVER_CHECKPOINT)
    .trigger(availableNow = True)
    .start(SILVER_DELTA)
)

print("Write stream to Silver Delta table started as silver_stream.")

In [0]:
silver_stream.awaitTermination()

In [0]:
silver_stream.status

In [0]:
spark.readStream.load(SILVER_DELTA).createOrReplaceTempView('silver_temp')

In [0]:
%sql
SELECT COUNT(*) FROM silver_temp

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# Load the UDF from MLflow
sentiment_analysis_udf = mlflow.pyfunc.spark_udf(spark, "models:/HF_TWEET_SENTIMENT/Production")


In [0]:
# Create a streaming DataFrame from the Silver Delta table
raw_silver_stream = spark.readStream.format("delta").load(SILVER_DELTA)

In [0]:
display(raw_silver_stream)

In [0]:
raw_gold_stream = (raw_silver_stream
    .withColumn("model_output", sentiment_analysis_udf(col("cleaned_text")))
    .select(
        col("timestamp"),
        col("mention"),
        col("cleaned_text"),
        col("sentiment"),
        col("model_output.label").alias("predicted_sentiment"),
        col("model_output.score").alias("predicted_score"),
        when(col("sentiment") == "negative", -1).
        when(col("sentiment") == "positive", 1).otherwise(0).alias("sentiment_id"),
        when(col("model_output.label") == "NEG", 0)
        .when(col("model_output.label") == "POS", 1).otherwise(-1)
        .alias("predicted_sentiment_id")
    )
    .withColumn("predicted_score", col("predicted_score")*100)
)

In [0]:
# Start the streaming write to the Gold Delta table
gold_stream = (
    raw_gold_stream.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", GOLD_CHECKPOINT)
    .trigger(availableNow = True)
    .start(GOLD_DELTA)
)


In [0]:
gold_stream.awaitTermination()

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
gold_stream.awaitTermination()


In [0]:
# ENTER YOUR CODE HERE
gold_df = spark.read.format("delta").load(GOLD_DELTA)


In [0]:

with mlflow.start_run(run_name = "Twitter_Sentiment_Analysis") as run:
    class_labels = ["neutral", "negative", "positive"]

    actual_sentiments = gold_df.select("sentiment_id").toPandas()["sentiment_id"]
    predicted_sentiments = gold_df.select("predicted_sentiment_id").toPandas()["predicted_sentiment_id"]

    # Calculate precision, recall, and F1-score
    precisions = precision_score(actual_sentiments, predicted_sentiments, average=None)
    recalls = recall_score(actual_sentiments, predicted_sentiments, average=None)
    f1_scores = f1_score(actual_sentiments, predicted_sentiments, average=None)

    # Log precision, recall, and F1-score as MLflow metrics
    for sentiment, precision, recall, f1_score_value in zip(class_labels, precisions, recalls, f1_scores):
        mlflow.log_metric(f"precision_{sentiment}", precision)
        mlflow.log_metric(f"recall_{sentiment}", recall)
        mlflow.log_metric(f"f1_score_{sentiment}", f1_score_value)

    # Create the confusion matrix
    confusion_matrix = pd.crosstab(actual_sentiments, predicted_sentiments, dropna = True)

    # Plot and save the confusion matrix as an image
    plt.figure(figsize=(6, 4))
    sns.heatmap(confusion_matrix, annot=True, cmap="Reds", fmt="d")
    plt.xlabel("Predicted Sentiment")
    plt.ylabel("Actual Sentiment")
    plt.savefig("confusion_matrix.png")

    # Log the confusion matrix image as an MLflow artifact
    mlflow.log_artifact("confusion_matrix.png")

    mlflow.log_param("Model Name", MODEL_NAME)
    mlflow.log_param("Model Version", "1")
    
    
    delta_table = DeltaTable.forPath(spark, SILVER_DELTA)
    table_version = delta_table.history().select("version").limit(1).collect()[0][0]

    mlflow.log_param("Silver Table Version", str(table_version))

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
  
  Solution: 97455 
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
spark.readStream.load(GOLD_DELTA).createOrReplaceTempView('gold_temp')

In [0]:
%sql
SELECT COUNT(*) FROM gold_temp

In [0]:
%sql
SELECT mention,
    COUNT(*) AS total_mentions,
    SUM(CASE WHEN predicted_sentiment_id = '1' THEN 1 ELSE 0 END) AS positive_mentions,
    SUM(CASE WHEN predicted_sentiment_id = '-1' THEN 1 ELSE 0 END) AS negative_mentions,
    SUM(CASE WHEN predicted_sentiment_id = '0' THEN 1 ELSE 0 END) AS neutral_mentions,
    (SUM(CASE WHEN predicted_sentiment_id = '1' THEN 1 ELSE 0 END) + 
     SUM(CASE WHEN predicted_sentiment_id = '-1' THEN 1 ELSE 0 END) + 
     SUM(CASE WHEN predicted_sentiment_id = '0' THEN 1 ELSE 0 END)) AS total_count
FROM gold_temp
GROUP BY mention
ORDER BY total_mentions DESC

In [0]:
%sql
SELECT
    mention,
    SUM(CASE WHEN predicted_sentiment_id = '1' THEN 1 ELSE 0 END) AS positive_mentions
FROM gold_temp
GROUP BY mention
ORDER BY positive_mentions DESC
LIMIT 20

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
    mention,
    SUM(CASE WHEN predicted_sentiment_id = '-1' THEN 1 ELSE 0 END) AS negative_mentions
FROM gold_temp
GROUP BY mention
ORDER BY negative_mentions DESC
LIMIT 20

Databricks visualization. Run in Databricks to view.

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
for stream in spark.streams.active:
    print(stream.name, " is stopped.")
stop_all_streams()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()
print("Elapsed time: ", END_TIME - START_TIME)

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

1. Data Spill: The system shows no data spillage, as evidenced by the uniformity in partition sizes and the small size of the source files. This ensures optimal use of disk resources and prevents I/O bottlenecks during data processing.
2. Data Skew: Absence of data skew is confirmed through the consistent file sizes and the effective application of Adaptive Query Execution (AQE), which dynamically optimizes the distribution of data across partitions. This balance eliminates the risk of overloading specific nodes, thereby enhancing overall job performance.
3. Shuffle Operations: Extensive shuffle operations are necessitated by complex transformations such as GroupBy, OrderBy, Aggregate, and Explode. Optimally configuring the shuffle partitions to align with the number of available processor cores minimizes cross-partition data transfer and reduces shuffle-related latency.
4. Storage Optimization: The initial storage of JSON files in the S3 bucket is characterized by a high count of small files, leading to inefficient read operations. Improvements are necessary at the ingestion stage, possibly through pre-processing steps that consolidate files into larger units for more efficient batch reading.
5. Serialization Efficiency: The serialization of the ML model into a deployable format via a User Defined Function (UDF) is identified as the most time-consuming step in the workflow. Given the inefficiencies of UDFs in Spark, exploring alternative built-in Spark functions for equivalent tasks could yield significant performance gains, reducing both execution time and computational overhead.